<a href="https://colab.research.google.com/github/lkodwani/ITCS-6112-Group-Project/blob/main/crime_rate_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Spark Environment Setup

In [3]:
#pyspark setup

!apt-get update
# Install Java 8 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Install Spark
!pip install pyspark

# setup environment variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,508 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

# 0.1 Crime Data Generation

In [4]:
import pyspark
from pyspark.sql import SparkSession

# Create or get an existing Spark session
spark = SparkSession.builder.master("local[*]").appName("Crime-rate-prediction").getOrCreate()

In [5]:
import pandas as pd
import numpy as np

def generate_city_crime_data(num_cities=100, seed=42):
    """
    Generates synthetic data for city crime rates with logical correlations and occasional null values.

    Args:
    - num_cities (int): Number of cities to generate data for.
    - seed (int): Random seed for reproducibility.

    Returns:
    - pd.DataFrame: A DataFrame containing the generated data.
    """
    np.random.seed(seed)

    # Generate city names
    cities = [f"City{i}" for i in range(1, num_cities + 1)]

    # Generate population density (people per square mile)
    population_density = np.random.randint(500, 20000, size=num_cities)

    # Poverty rate (%) - Higher in densely populated cities
    poverty_rate = np.clip(15 + (population_density / 2000) + np.random.normal(0, 5, num_cities), 5, 30)

    # Police budget ($) - Correlated with population density
    police_budget = (population_density * 50) + np.random.randint(500000, 2000000, size=num_cities)

    # Unemployment rate (%) - Higher in cities with high poverty
    unemployment_rate = np.clip(5 + (poverty_rate / 3) + np.random.normal(0, 2, num_cities), 3, 15)

    # Housing quality index (1 to 10) - Inversely related to poverty rate
    housing_quality_index = np.clip(10 - (poverty_rate / 5) + np.random.normal(0, 1, num_cities), 1, 10)

    # Traffic violation rate (per 1000 people) - Higher in densely populated areas
    traffic_violation_rate = np.clip(100 + (population_density / 100) + np.random.normal(0, 30, num_cities), 50, 500)

    # Drug abuse rate (%) - Higher in cities with low housing quality
    drug_abuse_rate = np.clip(10 + (20 - housing_quality_index) + np.random.normal(0, 5, num_cities), 5, 40)

    # Mental health services availability (1 to 10) - Better in cities with higher police budgets
    mental_health_services = np.clip(5 + (police_budget / 200000) + np.random.normal(0, 2, num_cities), 1, 10)

    # Gun ownership rate (%) - Randomly distributed but generally higher in cities with lower population density
    gun_ownership_rate = np.clip(20 + (15000 / (population_density + 1)) + np.random.normal(0, 10, num_cities), 20, 60)

    # Crime rate (per 1000 people) - Correlated with poverty, unemployment, and drug abuse
    crime_rate = np.clip(20 + (poverty_rate * 0.8) + (unemployment_rate * 1.2) +
                         (drug_abuse_rate * 0.5) - (mental_health_services * 2) +
                         np.random.normal(0, 10, num_cities), 10, 100)

    # Introduce occasional null values randomly
    for col in ['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']:
        null_indices = np.random.choice(num_cities, size=int(0.1 * num_cities), replace=False)
        locals()[col][null_indices] = np.nan

    # Create DataFrame
    data = pd.DataFrame({
        'city_name': cities,
        'population_density': population_density,
        'poverty_rate': poverty_rate,
        'police_budget': police_budget,
        'unemployment_rate': unemployment_rate,
        'housing_quality_index': housing_quality_index,
        'traffic_violation_rate': traffic_violation_rate,
        'drug_abuse_rate': drug_abuse_rate,
        'mental_health_services': mental_health_services,
        'gun_ownership_rate': gun_ownership_rate,
        'crime_rate': crime_rate
    })

    return data

# Generate and display the synthetic data
crime_data = generate_city_crime_data()

# Task 1: Data Preprocessing & Feature Engineering

Preview top 10 rows of data

In [5]:
crime_data.head(10)

,city_name,population_density,poverty_rate,police_budget,unemployment_rate,housing_quality_index,traffic_violation_rate,drug_abuse_rate,mental_health_services,gun_ownership_rate,crime_rate
0,City1,16295,27.321477,1317619,10.986045,NaN,301.213897,21.255302,10.000000,30.417632,66.092051
1,City2,1360,17.162807,1349474,14.721632,8.710644,110.853018,14.995146,8.745041,27.411433,46.255446
2,City3,5890,12.755851,1248851,6.851687,7.106241,187.012176,28.837764,10.000000,20.000000,34.298549
3,City4,12464,20.852981,2484028,14.353755,6.078718,207.560216,23.714740,10.000000,23.778102,62.900000
4,City5,11784,25.756818,1725784,13.885273,5.173155,230.251384,25.696677,10.000000,24.117530,60.217491
5,City6,6765,22.360477,1456717,12.735307,7.203198,207.912655,27.336162,10.000000,22.602482,44.876219
6,City7,17350,30.000000,2506348,15.000000,4.888984,195.236360,25.067063,10.000000,20.000000,43.592490
7,City8,4926,19.153906,784767,7.624010,6.665283,129.607441,25.591773,10.000000,21.829983,13.551667
8,City9,14923,30.000000,2282835,15.000000,4.055572,267.042265,30.023686,10.000000,43.789880,52.635186
9,City10,11863,16.329546,1970994,9.421293,6.794324,255.014452,26.039456,10.000000,20.000000,25.525565


Find out missing or null values in columns

In [7]:

missing_columns = crime_data.columns[crime_data.isnull().any()].tolist()

print(missing_columns)




['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']


**Task 1 - Handle Missing Data**

In [6]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# Convert pandas DataFrame to Spark DataFrame
crime_df = spark.createDataFrame(crime_data)

# Step 1: Handle Missing Values
crime_df = crime_df.na.fill({
    "housing_quality_index": crime_df.select(F.percentile_approx("housing_quality_index", 0.5)).collect()[0][0],
    "poverty_rate": crime_df.select(F.percentile_approx("poverty_rate", 0.5)).collect()[0][0],
    "traffic_violation_rate": crime_df.select(F.percentile_approx("traffic_violation_rate", 0.5)).collect()[0][0]
})

# Step 2: Feature Engineering
crime_df = crime_df.withColumn(
    "income_to_police_budget_ratio",
    F.col("population_density") / F.col("police_budget")
).withColumn(
    "unemployment_population_ratio",
    F.col("unemployment_rate") / F.col("population_density")
)

# Step 3: Detect and Cap Outliers
crime_df = crime_df.withColumn(
    "crime_rate", F.when(F.col("crime_rate") > 90, 90).otherwise(F.col("crime_rate"))
)

# Step 4: Assemble Features
assembler = VectorAssembler(
    inputCols=[
        "population_density", "poverty_rate", "police_budget",
        "unemployment_rate", "housing_quality_index",
        "traffic_violation_rate", "drug_abuse_rate",
        "mental_health_services", "gun_ownership_rate",
        "income_to_police_budget_ratio", "unemployment_population_ratio"
    ],
    outputCol="features"
)
crime_df = assembler.transform(crime_df)

# Preview the processed data
crime_df.show(5)


+---------+------------------+------------------+-------------+------------------+---------------------+----------------------+------------------+----------------------+------------------+------------------+-----------------------------+-----------------------------+--------------------+
|city_name|population_density|      poverty_rate|police_budget| unemployment_rate|housing_quality_index|traffic_violation_rate|   drug_abuse_rate|mental_health_services|gun_ownership_rate|        crime_rate|income_to_police_budget_ratio|unemployment_population_ratio|            features|
+---------+------------------+------------------+-------------+------------------+---------------------+----------------------+------------------+----------------------+------------------+------------------+-----------------------------+-----------------------------+--------------------+
|    City1|             16295|27.321477096152144|      1317619|10.986044887176178|    6.078718286650141|     301.2138974596086|21.255

In [ ]:
from pyspark.sql.functions import col, when, lit
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler

# Step 1: Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(crime_data)

# Step 2: Handle missing values
# Replace missing values in numerical columns with the median
numerical_cols = ['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']
for col_name in numerical_cols:
    median_value = crime_data[col_name].median()
    spark_df = spark_df.withColumn(
        col_name, when(col(col_name).isNull(), lit(median_value)).otherwise(col(col_name))
    )

# Show the updated data
spark_df.show(5)


# Step 1: Create income_to_police_budget_ratio
spark_df = spark_df.withColumn(
    "income_to_police_budget_ratio",
    (lit(100000) * (1 - col("poverty_rate") / 100)) / col("police_budget")
)

# Step 2: Create unemployment_population_ratio
spark_df = spark_df.withColumn(
    "unemployment_population_ratio",
    col("unemployment_rate") / col("population_density")
)

# Show the data with new features
spark_df.select("income_to_police_budget_ratio", "unemployment_population_ratio").show(5)

# Step 1: Cap outliers for 'crime_rate' column
crime_rate_upper_limit = 90  # Assume 90 as the maximum sensible value for crime_rate
spark_df = spark_df.withColumn(
    "crime_rate", when(col("crime_rate") > crime_rate_upper_limit, lit(crime_rate_upper_limit)).otherwise(col("crime_rate"))
)

# Show the capped data
spark_df.select("crime_rate").show(5)

from pyspark.ml.feature import VectorAssembler

# List of feature columns
feature_columns = [
    "population_density",
    "poverty_rate",
    "police_budget",
    "unemployment_rate",
    "housing_quality_index",
    "traffic_violation_rate",
    "drug_abuse_rate",
    "mental_health_services",
    "gun_ownership_rate",
    "income_to_police_budget_ratio",
    "unemployment_population_ratio"
]

# Assemble features into a single vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(spark_df)

# Show the data with features vector
assembled_df.select("features", "crime_rate").show(5, truncate=False)




In [19]:
from pyspark.sql.functions  import col, when, lit, mean as _mean, stddev as _stddev, expr,isnan, count
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler

# Step 1: Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(crime_data)

# Step 1: Handle Missing Values
numerical_cols = ['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']

# Check for missing data (for debugging/insights)
spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in spark_df.columns]).show()

# Impute missing values with median for numerical columns
for col_name in numerical_cols:
    median_value = crime_data[col_name].median()  # Calculate median using pandas
    spark_df = spark_df.withColumn(
        col_name,
        when(isnan(col(col_name)) | col(col_name).isNull(), lit(median_value)).otherwise(col(col_name))
    )


# Step 2: Create New Features
spark_df = spark_df.withColumn(
    "income_to_police_budget_ratio",
    (lit(100000) * (1 - col("poverty_rate") / 100)) / col("police_budget")
)

spark_df = spark_df.withColumn(
    "unemployment_population_ratio",
    col("unemployment_rate") / col("population_density")
)

# Assume a city size of 50 square miles for spatial density features
city_size = 50  # in square miles
spark_df = spark_df.withColumn(
    "crime_rate_per_sq_mile",
    col("crime_rate") / lit(city_size)
)

spark_df = spark_df.withColumn(
    "police_officer_per_sq_mile",
    (col("police_budget") / lit(100000)) / lit(city_size)
)

# Step 3: Handle Outliers
outlier_cols = ['housing_quality_index', 'drug_abuse_rate', 'crime_rate']
for col_name in outlier_cols:
    mean_stddev = spark_df.select(
        _mean(col_name).alias('mean'), _stddev(col_name).alias('stddev')
    ).first()
    upper_limit = mean_stddev['mean'] + 3 * mean_stddev['stddev']
    lower_limit = mean_stddev['mean'] - 3 * mean_stddev['stddev']
    spark_df = spark_df.withColumn(
        col_name, when(col(col_name) > upper_limit, lit(upper_limit))
                   .when(col(col_name) < lower_limit, lit(lower_limit))
                   .otherwise(col(col_name))
    )

# Step 4: Assemble Features
# Step 4: Assemble Features
assembler = VectorAssembler(
    inputCols=[
        "population_density", "poverty_rate", "police_budget",
        "unemployment_rate", "housing_quality_index",
        "traffic_violation_rate", "drug_abuse_rate",
        "mental_health_services", "gun_ownership_rate",
        "income_to_police_budget_ratio", "unemployment_population_ratio"
    ],
    outputCol="features"
)
spark_df = assembler.transform(spark_df)

# Final Dataset


spark_df.show(5)

+---------+------------------+------------+-------------+-----------------+---------------------+----------------------+---------------+----------------------+------------------+----------+
|city_name|population_density|poverty_rate|police_budget|unemployment_rate|housing_quality_index|traffic_violation_rate|drug_abuse_rate|mental_health_services|gun_ownership_rate|crime_rate|
+---------+------------------+------------+-------------+-----------------+---------------------+----------------------+---------------+----------------------+------------------+----------+
|        0|                 0|          10|            0|                0|                   10|                    10|              0|                     0|                 0|         0|
+---------+------------------+------------+-------------+-----------------+---------------------+----------------------+---------------+----------------------+------------------+----------+

+---------+------------------+------------------+